In [26]:
import sqlite3
import pandas as pd
import googlemaps
import yaml
import json
import datetime

In [3]:
# Connect to the database
conn = sqlite3.connect('../max-experiments/itinerary-scraping/journeys.db')
# display the schema
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

cursor.execute("SELECT * FROM journeys")

[('journeys',), ('stops',), ('outages',)]


In [25]:
# Sample Limit
SAMPLE_LIMIT = 176
# Get starting and ending coordinastes
cursor.execute("SELECT stops from journeys\n"
               "ORDER BY random()")
stops = cursor.fetchmany(SAMPLE_LIMIT)

# Load stop list and get start and end point
stop_dict = [json.loads(stop[0]) for stop in stops]
start_locations = [los[0]['stop_point'] for los in stop_dict]
end_locations = [los[-1]['stop_point'] for los in stop_dict]

print(f"Route Example: {start_locations[0]}->{end_locations[0]}")

# Get stop coordinate table prepared
cursor.execute("SELECT name,coord FROM stops")
stop_coords = dict(cursor.fetchall())

gmap_start_coords = []
gmap_end_coords = []

# Linearly (O(n) * O(1) Hashmap) maps all location coordinates to locations
for i in range(len(start_locations)):
    start = json.loads(stop_coords[start_locations[i]])
    end = json.loads(stop_coords[end_locations[i]])
    gmap_start_coords.append(str(f'{start["lat"]},{start["lon"]}'))
    gmap_end_coords.append(str(f'{end["lat"]},{end["lon"]}'))

print(f"Route Example Coordinates: ({gmap_start_coords[0]})->({gmap_end_coords[0]})")

# Save results
with open('data/start_coords.json','w') as f:
    json.dump(gmap_start_coords,f)
    f.close()

with open('data/end_coords.json','w') as f:
    json.dump(gmap_end_coords,f)
    f.close()

Route Example: Monplaisir La Plaine->Parc Tête d'Or Duquesne
Route Example Coordinates: (45.726309,4.88015)->(45.773249,4.850677)


In [27]:
# Initialize Google Maps API
# Get API Key from api_key.yaml
key = None
with open("api_key.yaml") as key_file:
    try:
        data = yaml.safe_load(key_file)
        key = data['key']
    except yaml.YAMLError as e:
        print(e)
        exit(1)

# Start Google Maps
gmaps = googlemaps.Client(key=key)

In [32]:
# Start requesting directions
directions_result_driving = gmaps.directions(gmap_start_coords[0],gmap_end_coords[0],mode="driving",departure_time=datetime.datetime.now())

directions_result_walking = gmaps.directions(gmap_start_coords[0],gmap_end_coords[0],mode="walking",departure_time=datetime.datetime.now())

def get_leg_dist(legs):
    distance = 0
    for leg in legs:
        distance += leg['distance']['value']
    return distance

# Calculate Walking amount statistic
if directions_result_driving and directions_result_walking:
    print(directions_result_driving[0]['legs'])

    driving_legs = directions_result_driving[0]['legs']
    walking_legs = directions_result_walking[0]['legs']

    driving_dist = get_leg_dist(driving_legs)
    walking_dist = get_leg_dist(walking_legs)

    print(f"Total Walking Distance: {walking_dist-driving_dist}")

[{'distance': {'text': '6.5 km', 'value': 6485}, 'duration': {'text': '20 mins', 'value': 1208}, 'duration_in_traffic': {'text': '16 mins', 'value': 956}, 'end_address': "Parc Tête d'Or Duquesne, 69006 Lyon, France", 'end_location': {'lat': 45.7732362, 'lng': 4.8506796}, 'start_address': 'Monplaisir La Plaine, 69008 Lyon, France', 'start_location': {'lat': 45.7263257, 'lng': 4.8801766}, 'steps': [{'distance': {'text': '0.8 km', 'value': 778}, 'duration': {'text': '2 mins', 'value': 145}, 'end_location': {'lat': 45.7316996, 'lng': 4.8738145}, 'html_instructions': 'Head <b>northwest</b> on <b>Av. Paul Santy</b> toward <b>Rue de la Moselle</b>', 'polyline': {'points': 'q|avGcdx\\ILA@u@lAyAtCMTo@fAMP[^c@f@[^KN{@hAKNcAhAKLw@|@OPKNmAvAe@d@WTeBfBe@h@EB?@kApAy@`Ak@h@OJUT'}, 'start_location': {'lat': 45.7263257, 'lng': 4.8801766}, 'travel_mode': 'DRIVING'}, {'distance': {'text': '1.3 km', 'value': 1296}, 'duration': {'text': '4 mins', 'value': 254}, 'end_location': {'lat': 45.7433419, 'lng': 4.